# 畳み込みニューラルネットワーク (CNN) 
CNN は画像認識の分野で非常によく使われています。
ここでは CNNをPyTorchで実装して、手書き文字認識(MNIST)の問題を解いてみます。


In [ ]:
# PyTorchが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1

from torch import set_num_threads, set_num_interop_threads
num_threads = 1
set_num_threads(num_threads)
set_num_interop_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## MNIST データセットのインポート

In [ ]:
# MNIST データセットのインポート
from torchvision import datasets, transforms
train_dataset = datasets.MNIST('/data/staff/deeplearning/datasets_pytorch', train=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST('/data/staff/deeplearning/datasets_pytorch', train=False, transform=transforms.ToTensor())

`train_dataset`は学習用データセット、`test_dataset`はモデル評価用データセットです。
それぞれのデータセットのエントリーは以下のようにして確認できます。

In [ ]:
image_0th, label_0th = train_dataset[0]
print('image shape = ', image_0th.shape)
print('label = ', label_0th)

ミニバッチの実装を簡単にするため、DataLoaderを使用します。

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=100, shuffle=True)

DataLoaderを使うと、以下のように指定したバッチサイズでデータが簡単に取得できます。

In [ ]:
for images, labels in train_dataloader:
    # for文で指定したバッチサイズ(ここでは100)ごとにデータが切り出されます。
    print(f'image shape = {images.shape}, labels shape = {labels.shape}')
    break

MNISTのトレーニング用データは6万画像ありますが、このノートブック内では計算時間を短くするため、6000画像だけを使うことにします。

In [ ]:
from torch.utils.data import Subset
train_dataloader = DataLoader(Subset(train_dataset, np.arange(6000)), batch_size=100, shuffle=True)

## MNIST 画像の表示
画像と、それに対応するラベルを見てみます。

In [ ]:
index = 0
plt.imshow(train_dataset[index][0].squeeze())
plt.show()
print(f'label = {train_dataset[index][1]}')

index = 1
plt.imshow(train_dataset[index][0].squeeze())
plt.show()
print(f'label = {train_dataset[index][1]}')

index = 2
plt.imshow(train_dataset[index][0].squeeze())
plt.show()
print(f'label = {train_dataset[index][1]}')

## CNN モデルの定義

In [ ]:
from torch.nn import Sequential
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Flatten
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax

model = Sequential(
    Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3)),
    MaxPool2d(kernel_size=(2, 2)),
    Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3)),
    MaxPool2d(kernel_size=(2, 2)),
    Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3)),
    Flatten(),
    Linear(in_features=576, out_features=64),
    ReLU(),
    Linear(in_features=64, out_features=10),
    # Softmax(dim=1)  # PyTorchではロス関数に CrossEntropyLoss を指定すると、自動でSoftmaxが適用されます。そのため、モデルにSoftmaxを適用する必要はありません。
)

from torchinfo import summary
summary(
    model,
    input_size=next(iter(train_dataloader))[0].shape,
    col_names=['output_size', 'num_params']
)

### トレーニング

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
loss_fn = CrossEntropyLoss()
optimizer = Adam(model.parameters())

# トレーニング
num_epochs = 10
for i_epoch in range(num_epochs):

    # エポックごとのロス、accuracyを計算するための変数
    loss_total = 0.
    accuracy_total = 0.

    for images, labels in train_dataloader:
        model.train()

        # 順伝搬
        y_pred = model(images)

        # ロスの計算
        loss = loss_fn(y_pred, labels)
        loss_total += loss.detach().numpy()

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()

        # 正解率(Accuracy)
        label_pred = y_pred.max(dim=1)[1]
        accuracy_total += (label_pred == labels).sum().numpy() / len(images)
    
    # ロス、accuracyをミニバッチの数で割って平均を取ります。
    loss_total /= len(train_dataloader)
    accuracy_total /= len(train_dataloader)
    print(f'epoch = {i_epoch}, loss = {loss_total}, acc = {accuracy_total}')


### 性能評価
性能評価用のデータセット(`test_dataloader`)を使って性能評価してみます。

In [ ]:
# モデルの性能評価
loss_total = 0
accuracy_total = 0.

model.eval()
for images, labels in test_dataloader:
    # 順伝搬
    y_pred = model(images)

    # ロスの計算
    loss = loss_fn(y_pred, labels)
    loss_total += loss.detach().numpy()

    # 正解率(Accuracy)の計算
    label_pred = y_pred.max(dim=1)[1]
    accuracy_total += (label_pred == labels).sum().numpy() / len(images)

loss_total /= len(test_dataloader)
accuracy_total /= len(test_dataloader)
print(f'loss = {loss_total}, acc = {accuracy_total}')


accuracy が 95%以上と、良い精度で判別ができていると思います。
間違った画像がどのようなものかも確認してみましょう。

In [ ]:
model.eval()

import torch
for images, labels in test_dataloader:
    y_pred = model(images)
    label_pred = y_pred.max(dim=1)[1]

    # 予測値と正解ラベルが一致していないエントリーを抽出します。
    wrong_image_indices = (label_pred != labels).nonzero().numpy()

    for index in wrong_image_indices:
        plt.imshow(images[index].squeeze())
        plt.show()
        print(f'label = {labels[index].squeeze().numpy()}')
        print(f'prediction = {label_pred[index].squeeze().numpy()}')
    break


## (おまけ) CNNとMLPの比較
MNIST を MLPで解くとどうなるかも調べてみましょう。

In [ ]:
from torch.nn import Sequential
from torch.nn import Flatten
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

# モデルの定義
model_dnn = Sequential(
    Flatten(),  # 画像を1次元のベクトルに変換: 28 * 28 = 784
    Linear(in_features=784, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=10),  # ノード数が10の層を追加。
)

loss_fn = CrossEntropyLoss()
optimizer = Adam(model_dnn.parameters())

# トレーニング
num_epochs = 10
for i_epoch in range(num_epochs):
    loss_total = 0.
    accuracy_total = 0.
    for images, labels in train_dataloader:
        model_dnn.train()

        # 順伝搬
        y_pred = model_dnn(images)

        # ロスの計算
        loss = loss_fn(y_pred, labels)
        loss_total += loss.detach().numpy()

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()

        # 正解率(Accuracy)
        label_pred = y_pred.max(dim=1)[1]
        accuracy_total += (label_pred == labels).sum().numpy() / len(images)
    
    # ロス、accuracyをミニバッチの数で割って平均を取ります。
    loss_total /= len(train_dataloader)
    accuracy_total /= len(train_dataloader)
    print(f'epoch = {i_epoch}, loss = {loss_total}, acc = {accuracy_total}')

# モデルの性能評価
loss_total = 0
accuracy_total = 0.

model_dnn.eval()
for images, labels in test_dataloader:
    # 順伝搬
    y_pred = model_dnn(images)

    # ロスの計算
    loss = loss_fn(y_pred, labels)
    loss_total += loss.detach().numpy()

    # 正解率(Accuracy)の計算
    label_pred = y_pred.max(dim=1)[1]
    accuracy_total += (label_pred == labels).sum().numpy() / len(images)

loss_total /= len(test_dataloader)
accuracy_total /= len(test_dataloader)
print(f'test loss = {loss_total}, test acc = {accuracy_total}')


どのくらいの精度が出たでしょうか？

次は、画像のピクセルのシャッフルをしてみます。

これを画像としてプロットすると、人間には理解不能なものになっていることがわかります。


In [ ]:
image = train_dataset[0][0]

# 全ての画像に対して、同じルールでピクセルのシャッフルをします。
permute = np.random.permutation(28 * 28)
image = image.numpy().flatten()[permute].reshape([1, 28, 28])

plt.imshow(image.squeeze())
plt.show()

これをCNN, MLPで学習させると、どうなるでしょうか？

In [ ]:
# CNN の学習
from torch.nn import Sequential
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Flatten
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

model_cnn = Sequential(
    Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3)),
    MaxPool2d(kernel_size=(2, 2)),
    Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3)),
    MaxPool2d(kernel_size=(2, 2)),
    Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3)),
    Flatten(),
    Linear(in_features=576, out_features=64),
    ReLU(),
    Linear(in_features=64, out_features=10),
)

loss_fn = CrossEntropyLoss()
optimizer = Adam(model_cnn.parameters())

# トレーニング
num_epochs = 10
for i_epoch in range(num_epochs):
    loss_total = 0.
    accuracy_total = 0.
    for images, labels in train_dataloader:
        model_cnn.train()

        # 全ての画像に対して、同じルールでピクセルのシャッフルをします。
        images = images.flatten(start_dim=2)[:, :, permute].reshape([-1, 1, 28, 28])

        # 順伝搬
        y_pred = model_cnn(images)

        # ロスの計算
        loss = loss_fn(y_pred, labels)
        loss_total += loss.detach().numpy()

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()

        # 正解率(Accuracy)
        label_pred = y_pred.max(dim=1)[1]
        accuracy_total += (label_pred == labels).sum().numpy() / len(images)
    
    # ロス、accuracyをミニバッチの数で割って平均を取ります。
    loss_total /= len(train_dataloader)
    accuracy_total /= len(train_dataloader)
    print(f'epoch = {i_epoch}, loss = {loss_total}, acc = {accuracy_total}')


# モデルの性能評価
loss_total = 0
accuracy_total = 0.

model_cnn.eval()
for images, labels in test_dataloader:
    # 全ての画像に対して、同じルールでピクセルのシャッフルをします。
    images = images.flatten(start_dim=2)[:, :, permute].reshape([-1, 1, 28, 28])

    # 順伝搬
    y_pred = model_cnn(images)

    # ロスの計算
    loss = loss_fn(y_pred, labels)
    loss_total += loss.detach().numpy()

    # 正解率(Accuracy)の計算
    label_pred = y_pred.max(dim=1)[1]
    accuracy_total += (label_pred == labels).sum().numpy() / len(images)

loss_total /= len(test_dataloader)
accuracy_total /= len(test_dataloader)
print(f'test loss = {loss_total}, test acc = {accuracy_total}')


In [ ]:
from torch.nn import Sequential
from torch.nn import Flatten
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

# モデルの定義
model_dnn = Sequential(
    Flatten(),  # 画像を1次元のベクトルに変換: 28 * 28 = 784
    Linear(in_features=784, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=10),  # ノード数が10の層を追加。
)

loss_fn = CrossEntropyLoss()
optimizer = Adam(model_dnn.parameters())

# トレーニング
num_epochs = 10
for i_epoch in range(num_epochs):
    loss_total = 0.
    accuracy_total = 0.
    for images, labels in train_dataloader:
        model_dnn.train()

        # 全ての画像に対して、同じルールでピクセルのシャッフルをします。
        images = images.flatten(start_dim=2)[:, :, permute].reshape([-1, 1, 28, 28])

        # 順伝搬
        y_pred = model_dnn(images)

        # ロスの計算
        loss = loss_fn(y_pred, labels)
        loss_total += loss.detach().numpy()

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()

        # 正解率(Accuracy)
        label_pred = y_pred.max(dim=1)[1]
        accuracy_total += (label_pred == labels).sum().numpy() / len(images)
    
    # ロス、accuracyをミニバッチの数で割って平均を取ります。
    loss_total /= len(train_dataloader)
    accuracy_total /= len(train_dataloader)
    print(f'epoch = {i_epoch}, loss = {loss_total}, acc = {accuracy_total}')

# モデルの性能評価
loss_total = 0
accuracy_total = 0.

model_dnn.eval()
for images, labels in test_dataloader:
    # 全ての画像に対して、同じルールでピクセルのシャッフルをします。
    images = images.flatten(start_dim=2)[:, :, permute].reshape([-1, 1, 28, 28])

    # 順伝搬
    y_pred = model_dnn(images)

    # ロスの計算
    loss = loss_fn(y_pred, labels)
    loss_total += loss.detach().numpy()

    # 正解率(Accuracy)の計算
    label_pred = y_pred.max(dim=1)[1]
    accuracy_total += (label_pred == labels).sum().numpy() / len(images)

loss_total /= len(test_dataloader)
accuracy_total /= len(test_dataloader)
print(f'test loss = {loss_total}, test acc = {accuracy_total}')


画像のピクセルをシャッフルする前と比べて、CNN/DNNの性能はどのように変化したでしょうか？